In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 19 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,515 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [2]:
!sudo apt-get install tesseract-ocr-fas


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-fas
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 301 kB of archives.
After this operation, 446 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-fas all 1:4.00~git30-7274cfa-1.1 [301 kB]
Fetched 301 kB in 1s (587 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tesseract-ocr-fas.
(Reading database

In [3]:
import pytesseract
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image

In [4]:
!pip install python-telegram-bot==13.3
!pip install pip install telethon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.3/436.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.8/563.8 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for telethon: filename=Telethon-1.32.0-py3-none-any.whl size=635342 sha256=b32104410d92e683037ba74506d6a0c93997b4411e869acd35a093aa6bd35e61
  Stored in directory: /root/.cache/pip/wheels/bf/53/13/6e6a1d98fbcb84303a86251aace79b7df7d5efb1fa7d8de37d
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=b533d42bf12e588baf7dfdecbb84a10aec853002ab655b6851e5ced30a006e29
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built telethon pyaes


In [5]:
!pip install pdf2image

In [1]:

# )))))))))))))))))))))))))))))))))))))))))))))
import os
import re

import cv2
import numpy as np
import pytesseract
from PIL import Image


def process_image(img_path):
    temp_filename = resize_image(img_path)
    img = remove_noise_and_smooth(temp_filename)
    img = fix_rotation(img)
    # img = remove_lines(img)

    return img


def resize_image(img_path):
    try:
        img = Image.open(img_path)
        length_x, width_y = img.size
        factor = max(1, int(1800 / length_x))  # 1800 for tesserect
        size = factor * length_x, factor * width_y
        im_resized = img.resize(size, Image.ANTIALIAS)

        import tempfile
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".TIFF")
        temp_filename = temp_file.name
        im_resized.save(temp_filename, dpi=(300, 300))  # best for OCR

        return temp_filename
    except IOError:
        print("Error while reading the file.")


def remove_noise_and_smooth(img_path):
    try:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Applying erosion and dilation to remove the noise
        # img = cv2.bitwise_not(img)
        # kernel = np.ones((5, 5), np.uint8)
        # img = cv2.erode(img, kernel, iterations=1)
        # img = cv2.dilate(img, kernel, iterations=1)
        # show_wait_destroy('dilate', img)
        # img = cv2.bitwise_not(img)

        kernel = np.ones((1, 1), np.uint8)
        opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        img = cv2.bitwise_or(img, closing)
        show_wait_destroy('bitwise_or', img)

        # img = apply_threshold(img, 1)
        # show_wait_destroy('threshold', img)
        # img = smooth_image(img)

        return img
    except IOError:
        print("Error while reading the file.")


def apply_threshold(img, argument):
    switcher = {
        1: cv2.threshold(cv2.GaussianBlur(img, (9, 9), 0), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        2: cv2.threshold(cv2.GaussianBlur(img, (7, 7), 0), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        3: cv2.threshold(cv2.GaussianBlur(img, (5, 5), 0), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        4: cv2.threshold(cv2.medianBlur(img, 5), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        5: cv2.threshold(cv2.medianBlur(img, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
        6: cv2.adaptiveThreshold(cv2.GaussianBlur(img, (5, 5), 0), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                 cv2.THRESH_BINARY, 31, 2),
        7: cv2.adaptiveThreshold(cv2.medianBlur(img, 3), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31,
                                 2),
    }
    return switcher.get(argument, "Invalid method")


def smooth_image(img):
    # Apply blur to smooth out the edges
    blur_img = cv2.GaussianBlur(img, (1, 1), 0)
    show_wait_destroy('blur', blur_img)

    return blur_img


def fix_rotation(img):
    rotated_img = img
    # osd: orientation and script detection
    tess_data = pytesseract.image_to_osd(img, nice=1)
    angle = int(re.search(r"(?<=Rotate: )\d+", tess_data).group(0))
    print("angle: " + str(angle))

    if angle != 0 and angle != 360:
        (h, w) = img.shape[:2]
        center = (w / 2, h / 2)

        # Perform the rotation
        rotation_mat = cv2.getRotationMatrix2D(center, -angle, 1.0)

        # Fixing the image cut-off by calculating the new center
        abs_cos = abs(rotation_mat[0, 0])
        abs_sin = abs(rotation_mat[0, 1])

        bound_w = int(h * abs_sin + w * abs_cos)
        bound_h = int(h * abs_cos + w * abs_sin)

        rotation_mat[0, 2] += bound_w / 2 - center[0]
        rotation_mat[1, 2] += bound_h / 2 - center[1]

        rotated_img = cv2.warpAffine(img, rotation_mat, (bound_w, bound_h))

    return rotated_img


def remove_lines(img):
    # Remove lines to improve accuracy of tabular documents
    # https://stackoverflow.com/questions/33949831/whats-the-way-to-remove-all-lines-and-borders-in-imagekeep-texts-programmatic?answertab=votes#tab-top
    result = img.copy()
    thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Remove horizontal lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
    remove_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
    cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(result, [c], -1, (255, 255, 255), 5)

    # Remove vertical lines
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
    remove_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
    cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(result, [c], -1, (255, 255, 255), 5)

    show_wait_destroy('nolines', result)

    return result


def show_wait_destroy(winname, img, active=False):
    if active:
        cv2.imshow(winname, cv2.resize(img, (960, 540)))
        cv2.moveWindow(winname, 500, 0)
        cv2.waitKey(0)
        cv2.destroyWindow(winname)


def save_image(img, img_path, method, active=False):
    if active:
        # Save the filtered image in the output directory
        filename = os.path.basename(img_path).split('.')[0]
        filename = filename.split()[0]
        save_path = os.path.join("../output", filename + "_filter_" + method + ".jpg")
        cv2.imwrite(save_path, img)

In [2]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters


def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Send me a photo of Persian text.")

def process_photo(update, context):
    photo_file = update.message.photo[-1].get_file()
    photo_file.download('photo.jpg')
    img = process_image('photo.jpg')
    config = ''
    text = str(pytesseract.image_to_string(img, lang='eng+fas', config=config))
    text = os.linesep.join([s for s in text.splitlines() if s.strip()])

    extracted_text = text
    print(extracted_text)
    context.bot.send_message(chat_id=update.effective_chat.id, text=extracted_text)

def main():
    updater = Updater(token='6382307368:AAHR2MWP1cZLFORkOZ0fOWxKwLQ-RP22AAs', use_context=True)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler('start', start))
    dp.add_handler(MessageHandler(Filters.photo, process_photo))
    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

<ipython-input-1-f4f2a156acf5>:25: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im_resized = img.resize(size, Image.ANTIALIAS)


angle: 0
موتور جستجوی فونت یاب در تاریخ ۱/۱/۹۱ پا به دنیای مجازی اینترنت گذاشت و با هدف رشد و ارتقاع سطح
گرافیک کشور فعالیت خود را شروع کرد .
در فونت یاب می توانید به صورت تکچین فونت ها را مشاهده و دریافت نمایید . در این روش با توجه به این
نکات که ۹۰.۰ فونت های پکیچ شده بکار نمی آید و ممکن است بیشتر فونت ها را شما قبلا داشته باشید
ارزش استفاده دارد .
